In [1]:
import cv2
import mediapipe as mp
import numpy as np
import torch
import torch.nn as nn
from scipy.spatial import distance

objc[2165]: Class CaptureDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x16cbe66b8) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x11e568860). One of the two will be used. Which one is undefined.
objc[2165]: Class CVWindow is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x16cbe6708) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1194fca68). One of the two will be used. Which one is undefined.
objc[2165]: Class CVView is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x16cbe6730) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_h

In [2]:
# MPS 사용 가능 여부 확인
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS")
else:
    device = torch.device("cpu")
    print("CPU")

MPS


In [ ]:
# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 랜드마크 인덱스 정의 (예: 코, 왼쪽 어깨, 오른쪽 어깨 등)
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]  # 총 11개 랜드마크

# Threshold 값 정의
threshold_normal = 8.5   # 일반 상태로 간주되는 속도 임계값
threshold_danger = 15.5   # 위험 상태로 간주되는 속도 임계값

def calculate_head_upper_body_speed(keypoints, prev_keypoints):
    h = np.array([keypoints[0, 0], keypoints[0, 1]])   # 머리 좌표
    l = np.array([keypoints[11, 0], keypoints[11, 1]])  # 왼쪽 어깨 좌표
    r = np.array([keypoints[12, 0], keypoints[12, 1]])  # 오른쪽 어깨 좌표

    # 이전 프레임의 좌표가 없는 경우 속도는 0으로 설정
    if prev_keypoints is None:
        return 0.0

    prev_h = np.array([prev_keypoints[0, 0], prev_keypoints[0, 1]])
    prev_l = np.array([prev_keypoints[11, 0], prev_keypoints[11, 1]])
    prev_r = np.array([prev_keypoints[12, 0], prev_keypoints[12, 1]])

    # 현재 프레임과 이전 프레임의 상체 중심 계산
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3

    # 유클리드 거리 계산 (속도)
    speed = distance.euclidean(center_new, center_prev)
    return speed

# GRU 모델 정의
class GRUModel(torch.nn.Module):
    def __init__(self, input_size=27):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size = 64
        self.num_layers = num_layers = 2
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,
                          dropout=0.5)
        self.fc = nn.Linear(hidden_size, 3)  # output_size를 직접 지정합니다.
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# GRU 모델 로드
input_size = 22
gru_model = GRUModel(input_size=input_size)  
gru_model.load_state_dict(torch.load('/Users/kimdeok-hwi/deeplearning/Project_humanFall/GRU/training_test_pts/2. mediapipe & sensordata/mediapipe_sensordata_except_normalization.pt', map_location=torch.device('cpu')))
gru_model.eval()

gru_model = gru_model.to(device)

# 클래스 이름 정의
class_names = {0: 'Normal', 1: 'Fall', 2: 'Danger'}

def process_landmarks(landmarks, bbox, speed):
    selected_landmarks = landmarks[LANDMARKS]   # 지정된 랜드마크 선택
    landmark_features = selected_landmarks[:, :2].flatten()  # (x,y) 좌표
    bbox_features = np.array(bbox).flatten()  # 바운딩 박스 좌표
    speed_feature = np.array([speed])  # 속도
    
    # 모든 특성을 결합
    features = np.concatenate([landmark_features, bbox_features, speed_feature])
    
    return features  # 총 27개의 특성 (22 + 4 + 1)

def calculate_and_draw_bbox(frame, landmarks):
    x_coordinates = landmarks[:, 0]
    y_coordinates = landmarks[:, 1]
    
    x1 = max(0, int(np.min(x_coordinates)))
    y1 = max(0, int(np.min(y_coordinates)))
    x2 = min(frame.shape[1], int(np.max(x_coordinates)))
    y2 = min(frame.shape[0], int(np.max(y_coordinates)))
    
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    
    # 바운딩 박스를 조금 더 넓게 조정 (각 방향으로 패딩 추가)
    padding = 50
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(frame.shape[1], x2 + padding)
    y2 = min(frame.shape[0], y2 + padding)

    return (x1, y1, x2, y2), bbox_width, bbox_height

# 낙상 감지 함수
def detect_fall(frame, landmarks, prev_landmarks, fall_frame_counter):
    global determine_fall, gru_model
    
    if determine_fall:
        return 1, fall_frame_counter
    
    speed = calculate_head_upper_body_speed(landmarks, prev_landmarks)
    bbox, bbox_width, bbox_height = calculate_and_draw_bbox(frame, landmarks)
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')
    
    processed_landmarks = process_landmarks(landmarks, bbox, speed)
    
    # GRU 모델 입력을 위한 데이터 준비
    input_data = torch.FloatTensor(processed_landmarks).unsqueeze(0).unsqueeze(0).to(device)
    
    # GRU 모델을 통한 예측
    with torch.no_grad():
        output = gru_model(input_data)
        _, predicted = torch.max(output.data, 1)
        bbox_class = predicted.item()
    
    # 후처리: 속도와 bbox_ratio를 기반으로 예측 결과 보정
    if speed < threshold_normal and bbox_ratio < 0.5:
        bbox_class = 0  # Normal
    elif speed >= threshold_danger or bbox_ratio > 1:
        bbox_class = 1  # Fall
    else:
        bbox_class = 2  # Danger
            
    print(f"Speed: {speed}, bbox_ratio: {bbox_ratio}")
    print(f"Final predicted class after post-processing: {bbox_class}")
    
    # Fall_counter 업데이트
    if bbox_class == 1:
        fall_frame_counter += 1
        if fall_frame_counter >= 10:
            determine_fall = True
    else:
        fall_frame_counter = 0

    return bbox_class, fall_frame_counter

cap = cv2.VideoCapture(0)

# 비디오 속성 가져오기 
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 60
cap.set(cv2.CAP_PROP_FPS, fps)

# 출력 비디오 설정 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_path = 'GRU_webcam_2.mp4'
out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

# 프레임 처리 루프 
fall_frame_counter = 0
determine_fall = False
prev_landmarks = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_pose = pose.process(rgb_frame)

    if results_pose.pose_landmarks:
        landmarks = np.array([[lm.x * width, lm.y * height, lm.z] for lm in results_pose.pose_landmarks.landmark])
       
        label, fall_frame_counter = detect_fall(frame, landmarks, prev_landmarks, fall_frame_counter)  
        print(f"Predicted Class: {label}")  

        # 바운딩 박스와 라벨 그리기 
        bbox, _, _ = calculate_and_draw_bbox(frame, landmarks)
        color = (0, 255, 0) if label == 0 else ((0, 255, 255) if label == 2 else (0, 0, 255)) 
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
        class_name = class_names[label] if label is not None else 'Unknown'
        cv2.putText(frame, f'GRU: {class_name}', (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        #if determine_fall: 
        #    cv2.putText(frame, 'FALL', (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 3)
        # 랜드마크 표시 
        mp_drawing.draw_landmarks(frame, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        prev_landmarks = landmarks

    # 프레임 저장 및 출력 
    resized_frame = cv2.resize(frame, (1920, 1080))  
    out.write(frame) 
    cv2.imshow('Fall Detection', resized_frame) 
    if cv2.waitKey(1) & 0xFF == ord('q'):
         break

cap.release()
out.release()
cv2.destroyAllWindows()

RuntimeError: input.size(-1) must be equal to input_size. Expected 22, got 27

: 